In [1]:
!python --version

Python 3.11.3


In [2]:
import os
import sys
import config

# load deps
from KMOpenAI import *

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
tiktok = Tokenizer()

def count_tokens(query):
    return tiktok.get_token_count(query)

In [4]:
os.environ['OPENAI_API_KEY'] = config.openai['key']

In [5]:
llm = OpenAIAPI(os.environ['OPENAI_API_KEY'])
result = llm.openai_completions("hello")
print(result['choices'][0]['message']['content'])

b'{"messages": [{"role": "user", "content": "hello"}], "temperature": 0.0, "top_p": 1.0, "frequency_penalty": 0.0, "presence_penalty": 0.0, "max_tokens": 1000, "model": "gpt-3.5-turbo", "stream": false}'
{
  "id": "chatcmpl-8xe7XbicWJKYsT7VTAn9BUB536eXG",
  "object": "chat.completion",
  "created": 1709226267,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 8,
    "completion_tokens": 9,
    "total_tokens": 17
  },
  "system_fingerprint": "fp_86156a94a0"
}

Hello! How can I assist you today?


In [6]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import OpenAI

In [7]:
llm = OpenAI()
llm.invoke("hello, what is 2+2")

'\n\n2+2 is equal to 4.'

In [259]:
chat_history = [
    {
        "role": "user",
        "content": "Hello! Who is the president of USA?"
    },
    {
        "role": "AI",
        "content": "The president of USA is Joe Biden."
    },
    {
        "role": "user",
        "content": "Nice. What's his height?"
    },
    {
        "role": "AI",
        "content": "It is 6 feet."
    },
    {
        "role": "user",
        "content": "Add 2 to this and convert it into cm."
    },
    {
        "role": "AI",
        "content": "It is now 243.84 centimeters."
    },
    {
        "role": "user",
        "content": "He belongs to what party?"
    },
    {
        "role": "AI",
        "content": "He belongs to Democratic Party of USA."
    },
    {
        "role": "user",
        "content": "What was the previous president Barack Obama's height?"
    },
    {
        "role": "AI",
        "content": "Barack Obama's height was 6.5 feet."
    },
    {
        "role": "user",
        "content": "Can you add 100 cms to my frist question person's height?"
    },
    {
        "role": "AI",
        "content": "Sure, Joe Biden's new height is 182.88 cms (6 feet) + 100 cms = 282.88 cms or 9.3 feet."
    },
    {
        "role": "user",
        "content": "Thank you."
    }
]

In [39]:
MAX_TOKEN_LIMIT = 200
MAX_SUMMARY_TOKEN_LIMIT = MAX_TOKEN_LIMIT - 100

In [57]:
chat_history_prompt_template = \
"""
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary within provided length of tokens.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary length:
20

New summary:
Human queries AI on AI's view. AI deems AI beneficial, enabling human potential...
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary length:
{length_of_summary}

New summary:
"""
chat_history_prompt_template_keywords = ["{summary}", "{new_lines}", "{length_of_summary}"]



qna_context_prompt_template = \
"""
You are a helpful AI assistant for Morgan Stanley users. Using conversational context provide answers to your best of capability.

EXAMPLE
Context:
The human asks what the is Barack Obama's height? The AI says its 6 feet.

Question:
Add 2 feet to his height.

Response:
Barack Obama's new height will be 8 feet.
END OF EXAMPLE

Context:
{chat_history}

Question:
{query}

Response:
"""
qna_context_prompt_template_keywords = ["{chat_history}", "{query}"]

In [58]:
llm = OpenAIAPI(os.environ['OPENAI_API_KEY'])

In [71]:
class ConversationRollingWindow():
    
    def __init__(self, max_token_limit, max_summary_token_limit):
        self.prompt_template = chat_history_prompt_template
        self.chat_history_prompt_template_keywords = chat_history_prompt_template_keywords        
        prompt_template_len = self.get_prompt_template_len()
        self.max_token_limit  = max_token_limit - prompt_template_len
        self.summary_len = max_summary_token_limit
        self.line_delimitter = "\n\n"
        
        self.new_lines = ""
        self.current_summary = ""

        self.conversation = []
        

    def get_prompt_template_len(self):
        template = chat_history_prompt_template
        for key in self.chat_history_prompt_template_keywords:
            template = template.replace(key, "")
        return count_tokens(template)
        
    def count_tokens(self, text):
        return tiktok.get_token_count(text)

    def extract_content(self, messages):
        try:
            if isinstance(messages, list):
                user_content = ""
                ai_content = ""
                for message in messages:
                    if message["role"] == "user":
                        user_content += message["content"] + "\n"
                    elif message["role"] == "AI":
                        ai_content += message["content"]
                return f"User: {user_content}AI: {ai_content}"
            else:
                return f"ERROR: pass a list of dict"
        except Exception as e:
            return f"ERROR: {str(e)}"

    def get_hat_history_context(self, conversation):
        if not isinstance(conversation, list):
            return ""
            
        # New message
        new_message_str = self.extract_content(conversation)
        count_new_message = self.count_tokens(new_message_str)
        print(f"<NEW_MESSAGE>\n{new_message_str} | Count: {count_new_message}\n")
        
        # New lines
        line_breaker = ""
        if len(self.new_lines) > 0:
            line_breaker = self.line_delimitter
        self.new_lines += line_breaker + new_message_str

        # Entire chat history
        conversation_history = (self.current_summary + "\n\n" + self.new_lines).strip("\n").strip()
        count_total_tokens = self.count_tokens(conversation_history)
        print(f"\n<CONVERSATION>\n{conversation_history} | Count: {count_total_tokens} | Limit: {self.max_token_limit}\n")

        if count_total_tokens >= self.max_token_limit:
            new_summary = self.chat_summarizer(self.new_lines)
            self.current_summary = new_summary.strip("\n").strip()
            self.new_lines = ""
        
        return f"{self.current_summary}\n\n{self.new_lines}".strip("\n").strip()

    def invoke_openai(self, prompt):
        result = llm.openai_completions(prompt)
        token_count = self.count_tokens(prompt)
        print(f"OpenAI Completions called: {token_count}")
        return result['choices'][0]['message']['content']
        
    def chat_summarizer(self, new_lines, summary_len=None):
        if not summary_len:
            summary_len = str(self.summary_len)
        summary_prompt = self.prompt_template \
                            .replace('{summary}', self.current_summary) \
                            .replace('{new_lines}', new_lines) \
                            .replace('{length_of_summary}', summary_len)
        print("\n+++++\n", summary_prompt, "\n++++\n")
        return self.invoke_openai(summary_prompt)


    def chat_qna(self, user_query):
        
        query = {"role": "user", "content": user_query}
        
        if len(self.conversation) == 0:
            prompt = qna_context_prompt_template.replace("{chat_history}", "").replace("{query}", user_query)
            result = self.invoke_openai(prompt)
            response = {"role": "AI", "content": result}
            self.conversation = [query, response]
        else:
            chat_history = self.get_hat_history_context(self.conversation)
            print("\n\n CHAT HISTORY RETRIEVED \n\n", chat_history)
            prompt = qna_context_prompt_template.replace("{chat_history}", chat_history).replace("{query}", user_query)
            result = self.invoke_openai(prompt)
            response = {"role": "AI", "content": result}

        return response


In [68]:
context_qna = ConversationRollingWindow(max_token_limit=MAX_TOKEN_LIMIT, max_summary_token_limit=MAX_SUMMARY_TOKEN_LIMIT)

In [69]:
context_qna.chat_qna('hi')

b'{"messages": [{"role": "user", "content": "\\nYou are a helpful AI assistant for Morgan Stanley users. Using conversational context provide answers to your best of capability.\\n\\nEXAMPLE\\nContext:\\nThe human asks what the is Barack Obama\'s height? The AI says its 6 feet.\\n\\nQuestion:\\nAdd 2 feet to his height.\\n\\nResponse:\\nBarack Obama\'s new height will be 8 feet.\\nEND OF EXAMPLE\\n\\nContext:\\n\\n\\nQuestion:\\nhi\\n\\nResponse:\\n"}], "temperature": 0.0, "top_p": 1.0, "frequency_penalty": 0.0, "presence_penalty": 0.0, "max_tokens": 1000, "model": "gpt-3.5-turbo", "stream": false}'
{
  "id": "chatcmpl-8xhJ4faleXgpGSH1qPTDWk7lpValu",
  "object": "chat.completion",
  "created": 1709238514,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 9

{'role': 'AI', 'content': 'Hello! How can I assist you today?'}

In [70]:
context_qna.chat_qna('whats ur name?')

<NEW_MESSAGE>
User: hi
AI: Hello! How can I assist you today? | Count: 15


<CONVERSATION>
User: hi
AI: Hello! How can I assist you today? | Count: 15 | Limit: 70

b'{"messages": [{"role": "user", "content": "\\nYou are a helpful AI assistant for Morgan Stanley users. Using conversational context provide answers to your best of capability.\\n\\nEXAMPLE\\nContext:\\nThe human asks what the is Barack Obama\'s height? The AI says its 6 feet.\\n\\nQuestion:\\nAdd 2 feet to his height.\\n\\nResponse:\\nBarack Obama\'s new height will be 8 feet.\\nEND OF EXAMPLE\\n\\nContext:\\nUser: hi\\nAI: Hello! How can I assist you today?\\n\\nQuestion:\\nwhats ur name?\\n\\nResponse:\\n"}], "temperature": 0.0, "top_p": 1.0, "frequency_penalty": 0.0, "presence_penalty": 0.0, "max_tokens": 1000, "model": "gpt-3.5-turbo", "stream": false}'
{
    "error": {
        "message": "Rate limit reached for gpt-3.5-turbo in organization org-A0JANTlqWEhEWrdT8VHDvUf2 on requests per min (RPM): Limit 3, Used 3, Reque

KeyError: 'choices'

In [37]:
context_qna = ConversationRollingWindow(max_token_limit=MAX_TOKEN_LIMIT, max_summary_token_limit=MAX_SUMMARY_TOKEN_LIMIT)

def qna(user_query):
    query = {"role": "user", "content": user_query}
    chat_history = context.get_hat_history_context(query)
    prompt = qna_context_prompt_template.replace("{chat_history}", chat_history).replace("{query}", user_query)
    result = context.invoke_openai(prompt)
    response = {"role": "AI", "content": result}
    return [query, response]

In [38]:
res = qna("Hello")

NameError: name 'messages' is not defined

In [34]:
res

[{'role': 'user', 'content': 'Hello'},
 {'role': 'AI', 'content': 'Hello! How can I assist you today?'}]

In [379]:
chat_history = context.get_hat_history_context(user_query)
template = qna_prompt.replace("{chat_history}", chat_history).replace("{query}", user_query)

<NEW_MESSAGE>
ERROR: pass a dict or list | Count: 7


<CONVERSATION>
ERROR: pass a dict or list | Count: 7 | Limit: 70



In [348]:
convo = sum_chat.handle_conversation(chat_history[:2])
print("\n\n=====OUTPUT=====\n\n", convo, "\n=============\n")

<NEW_MESSAGE>
User: Hello! Who is the president of USA?
AI: The president of USA is Joe Biden. | Count: 21


<CONVERSATION>
User: Hello! Who is the president of USA?
AI: The president of USA is Joe Biden. | Count: 21 | Limit: 70



=====OUTPUT=====

 User: Hello! Who is the president of USA?
AI: The president of USA is Joe Biden. 



In [349]:
convo = sum_chat.handle_conversation(chat_history[2:4])
print("\n\n=====OUTPUT=====\n\n", convo, "\n=============\n")

<NEW_MESSAGE>
User: Nice. What's his height?
AI: It is 6 feet. | Count: 17


<CONVERSATION>
User: Hello! Who is the president of USA?
AI: The president of USA is Joe Biden.

User: Nice. What's his height?
AI: It is 6 feet. | Count: 38 | Limit: 70



=====OUTPUT=====

 User: Hello! Who is the president of USA?
AI: The president of USA is Joe Biden.

User: Nice. What's his height?
AI: It is 6 feet. 



In [350]:
convo = sum_chat.handle_conversation(chat_history[4:6])
print("\n\n=====OUTPUT=====\n\n", convo, "\n=============\n")

<NEW_MESSAGE>
User: Add 2 to this and convert it into cm.
AI: It is now 243.84 centimeters. | Count: 25


<CONVERSATION>
User: Hello! Who is the president of USA?
AI: The president of USA is Joe Biden.

User: Nice. What's his height?
AI: It is 6 feet.

User: Add 2 to this and convert it into cm.
AI: It is now 243.84 centimeters. | Count: 63 | Limit: 70



=====OUTPUT=====

 User: Hello! Who is the president of USA?
AI: The president of USA is Joe Biden.

User: Nice. What's his height?
AI: It is 6 feet.

User: Add 2 to this and convert it into cm.
AI: It is now 243.84 centimeters. 



In [351]:
convo = sum_chat.handle_conversation(chat_history[6:8])
print("\n\n=====OUTPUT=====\n\n", convo, "\n=============\n")

<NEW_MESSAGE>
User: He belongs to what party?
AI: He belongs to Democratic Party of USA. | Count: 18


<CONVERSATION>
User: Hello! Who is the president of USA?
AI: The president of USA is Joe Biden.

User: Nice. What's his height?
AI: It is 6 feet.

User: Add 2 to this and convert it into cm.
AI: It is now 243.84 centimeters.

User: He belongs to what party?
AI: He belongs to Democratic Party of USA. | Count: 81 | Limit: 70


+++++
 
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary within provided length of tokens.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary length:
20

New summary:
Human queries AI on AI's view. AI deems AI benef

In [345]:
convo = sum_chat.handle_conversation(chat_history[8:10])
print("\n\n---\n\nCONVERSATION :::: \n\n", convo, "\n-----\n")

<NEW_MESSAGE>
User: What was the previous president Barack Obama's height?
AI: Barack Obama's height was 6.5 feet. | Count: 25


<CONVERSATION>
User inquires about USA president. AI responds with Joe Biden. User asks about his height, AI states 6 feet. User requests conversion to cm, AI provides 243.84 cm. User queries his political party, AI confirms Democratic Party.

User: What was the previous president Barack Obama's height?
AI: Barack Obama's height was 6.5 feet. | Count: 75 | Limit: 70


+++++
 
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary within provided length of tokens.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary len

In [300]:
convo = sum_chat.handle_conversation(chat_history[10:12])
print("\n\n---\n\nCONVERSATION :::: \n\n", convo, "\n-----\n")

New_Message: User: Can you add 100 cms to my frist question person's height?
AI: Sure, Joe Biden's new height is 182.88 cms (6 feet) + 100 cms = 282.88 cms or 9.3 feet. | Count: 53
Conversation:User asks about USA president, AI says Joe Biden. User asks height, AI says 6 feet. User converts to cm, AI provides 243.84 cm. User asks Biden's party, AI answers Democratic. User inquires about Obama's height, AI states 6.5 feet.User: Can you add 100 cms to my frist question person's height?
AI: Sure, Joe Biden's new height is 182.88 cms (6 feet) + 100 cms = 282.88 cms or 9.3 feet. | Count: 112 | Limit: 70

+++++
 
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary within provided length of tokens.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force 

In [89]:
summarized_conversation.prompt

AttributeError: 'str' object has no attribute 'prompt'

In [81]:
print(conversation_rolling_window.conversation_history)

User: Hello! Who is the president of USA?
AI: The president of USA is Joe Biden.


User: What is height?
AI: It's 6 feet.



In [40]:
summarized_conversation

'User: Hello! Who is the president of USA?\nAI: The president of USA is Joe Biden.\n'